<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Simple_Example_with_Ray_Tune_for_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ray

In [ ]:
pip install -U ray[tune]

In [ ]:
pip install --upgrade ray[tune]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms
from ray import tune
import ray

# Initialize Ray
ray.shutdown()  # Properly shut down Ray before initializing
ray.init(ignore_reinit_error=True)

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define training function
def train_mnist(config):
    try:
        # Load data
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
        train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
        train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)

        # Model, loss, optimizer
        model = SimpleNet(config["hidden_size"])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config["lr"])

        # Training loop
        for epoch in range(5):  # Run for a few epochs
            model.train()
            for images, labels in train_loader:
                images = images.view(images.size(0), -1)  # Flatten images
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Validation accuracy (using train set here for simplicity)
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for images, labels in train_loader:
                images = images.view(images.size(0), -1)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        return {"accuracy": accuracy}
    except Exception as e:
        print(f"Error occurred: {e}")
        return {"accuracy": 0.0}

# Ray Tune experiment setup
analysis = tune.run(
    train_mnist,
    config={
        "hidden_size": tune.choice([32, 64, 128]),
        "lr": tune.choice([0.001, 0.01, 0.1])
    },
    metric="accuracy",  # Specify the metric
    mode="max",         # Specify the mode
    num_samples=2,  # Reduced for testing
    resources_per_trial={"cpu": 1},
    verbose=1
)

print("Best config: ", analysis.best_config)

# Shutdown Ray
ray.shutdown()